In [1]:
cd "./diff_class_1/"

/Users/xuejiaxin/Dropbox/My Mac (Jiaxin的MacBook Pro)/Documents/GitHub/latent/pytorch-pretrained-BigGAN/diff_class_1


In [2]:
%load_ext autoreload
%autoreload 2

# target image
import torch
from pytorch_pretrained_biggan import (BigGAN, one_hot_from_names, truncated_noise_sample,
                                       save_as_images, display_in_terminal, one_hot_from_int)
import logging


logging.basicConfig(level=logging.INFO)

model = BigGAN.from_pretrained('biggan-deep-128')

truncation = 0.5
class_vector = one_hot_from_names(["dog"], batch_size=1)
noise_vector = truncated_noise_sample(truncation=truncation, batch_size=1, seed = 10)

noise_vector = torch.from_numpy(noise_vector)
class_vector = torch.from_numpy(class_vector)


with torch.no_grad():
    output = model(noise_vector, class_vector, truncation)

save_as_images(output, "dog_target")


INFO:pytorch_pretrained_biggan.model:loading model biggan-deep-128 from cache at /Users/xuejiaxin/.pytorch_pretrained_biggan/6371c3777477e4e75187da1b9b526561aac3134f38c7299a3438009ae560e20d.3434ebdfa74a8c17e0e56061cfd905fa163c92f110e88df77b47da6ce9910b48
INFO:pytorch_pretrained_biggan.model:Model config {
  "attention_layer_position": 8,
  "channel_width": 128,
  "class_embed_dim": 128,
  "eps": 0.0001,
  "layers": [
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      16
    ],
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      8
    ],
    [
      false,
      8,
      8
    ],
    [
      true,
      8,
      4
    ],
    [
      false,
      4,
      4
    ],
    [
      true,
      4,
      2
    ],
    [
      false,
      2,
      2
    ],
    [
      true,
      2,
      1
    ]
  ],
  "n_stats": 51,
  "num_classes": 1000,
  "output_dim": 128,
  "z_dim": 128
}

INFO:pytorch_pretrained_biggan.utils:Saving image to dog_target_

In [4]:
from torchvision.models import squeezenet1_0
from tqdm import trange
from torch import nn

DEVICE = 'cpu'
model = model.eval().to(DEVICE)


semantic_model = squeezenet1_0(pretrained=True).to(DEVICE)
semantic_model.classifier = torch.nn.Sequential(
    torch.nn.Flatten()
    )
semantic_model = semantic_model.eval()

soft_max = nn.Softmax(dim = 1)
cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)

truncation = 0.5
noise = truncated_noise_sample(truncation=truncation, batch_size=1, seed=10)
noise = torch.nn.Parameter(torch.tensor(noise, requires_grad=True).float().to(DEVICE))

class_vector = one_hot_from_names(['tabby cat'], batch_size=1)
class_vector = torch.nn.Parameter(torch.tensor(class_vector, requires_grad=True).float().to(DEVICE))

optim = torch.optim.Adam([noise, class_vector], lr=0.05)

save_as_images(model(noise, class_vector, truncation), f"start")

L = []
L_pixel = []
L_semantic = []

for iteration in trange(0, 301):
    optim.zero_grad()

    output_1 = torch.nn.functional.gumbel_softmax(class_vector, tau=1, hard=True)
    #output_1 = soft_max(class_vector)

    y_hat = model(noise, output_1, truncation)

    semantic_loss = -((semantic_model(y_hat) - semantic_model(output)) ** 2).mean() ** .5 
    #semantic_loss = -cos_sim(semantic_model(y_hat), semantic_model(output))
    L_semantic.append(semantic_loss.item())

    pixel_loss = abs(y_hat - output).mean()
    L_pixel.append(pixel_loss.item())

    loss = semantic_loss + 30 * pixel_loss
    L.append(loss.item())

    loss.backward()
    optim.step()


    if iteration % 1 == 0:
        print(output_1.argmax())
        save_as_images(y_hat, f"cat2dog_{iteration}")

INFO:pytorch_pretrained_biggan.utils:Saving image to start_0.png
  0%|          | 1/301 [00:02<12:22,  2.48s/it]tensor(118)
INFO:pytorch_pretrained_biggan.utils:Saving image to cat2dog_1_0.png
  1%|          | 2/301 [00:04<12:18,  2.47s/it]tensor(894)
INFO:pytorch_pretrained_biggan.utils:Saving image to cat2dog_2_0.png
  1%|          | 3/301 [00:07<12:16,  2.47s/it]tensor(558)
INFO:pytorch_pretrained_biggan.utils:Saving image to cat2dog_3_0.png
  1%|▏         | 4/301 [00:09<12:11,  2.46s/it]tensor(884)
INFO:pytorch_pretrained_biggan.utils:Saving image to cat2dog_4_0.png
  2%|▏         | 5/301 [00:12<12:10,  2.47s/it]tensor(710)
INFO:pytorch_pretrained_biggan.utils:Saving image to cat2dog_5_0.png
  2%|▏         | 6/301 [00:14<12:07,  2.47s/it]tensor(710)
INFO:pytorch_pretrained_biggan.utils:Saving image to cat2dog_6_0.png
  2%|▏         | 7/301 [00:17<12:03,  2.46s/it]tensor(422)
INFO:pytorch_pretrained_biggan.utils:Saving image to cat2dog_7_0.png
  3%|▎         | 8/301 [00:19<11:59,  2